Cell for google colab

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir('/content/drive/Othercomputers/My laptop/ML/YAGPT')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from config import params
from pprint import pprint
onServer = True

if not onServer:
    params['num_proc'] = 1
pprint(params)

{'additional_special_tokens': ['speaker1: ',
                               'speaker2: ',
                               'speaker3: ',
                               'speaker4: ',
                               'speaker5: ',
                               'speaker6: ',
                               'speaker7: ',
                               'speaker8: ',
                               'speaker9: ',
                               'history: '],
 'checkpoint': 'AlexWortega/instruct_rugptMedium',
 'data_path': 'data.csv',
 'dataset_name': 'under-tree/prepared-yagpt',
 'num_proc': 1}


In [3]:
!pip install transformers datasets huggingface_hub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
from datasets import load_dataset
from config import params
from pprint import pprint

name = params['dataset_name']
dataset = load_dataset(name, split='train')

In [6]:
# info of dataset
pprint(dataset.info)

DatasetInfo(description='', citation='', homepage='', license='', features={'input_ids': Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), 'attention_mask': Sequence(feature=Value(dtype='int8', id=None), length=-1, id=None)}, post_processed=None, supervised_keys=None, task_templates=None, builder_name=None, config_name=None, version=None, splits={'train': SplitInfo(name='train', num_bytes=2212959280, num_examples=840790, shard_lengths=[198158, 198158, 198158, 198158, 48158], dataset_name='parquet')}, download_checksums={'https://huggingface.co/datasets/under-tree/prepared-yagpt/resolve/662a5fd1c3c3fd5f12b50fb815238483b4ce0970/data/train-00000-of-00005-2ecd1f1a3f9993f4.parquet': {'num_bytes': 62369048, 'checksum': None}, 'https://huggingface.co/datasets/under-tree/prepared-yagpt/resolve/662a5fd1c3c3fd5f12b50fb815238483b4ce0970/data/train-00001-of-00005-d53e492257986516.parquet': {'num_bytes': 63735174, 'checksum': None}, 'https://huggingface.co/datasets/under-tree/pre

In [7]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer


checkpoint = params['checkpoint']
tokenizer = AutoTokenizer.from_pretrained(checkpoint, pad_token='<|pad|>')
n = 10
special_tokens = {'additional_special_tokens': params['additional_special_tokens']}
tokenizer.add_special_tokens(special_tokens)

model = AutoModelForCausalLM.from_pretrained(checkpoint)
model.resize_token_embeddings(len(tokenizer))
None

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
# Don't know why does it run for that long, so I'm not gonna launch it

In [9]:
# mx_len = max(len(x) for x in dataset['input_ids'])
# print(f'Max length: {mx_len}')

In [13]:
mx_len = 512
block_size = mx_len

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
        # customize this part to your needs.
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

dataset_prepared = dataset.map(group_texts, batched=True, num_proc=params['num_proc'])

Map (num_proc=10):   0%|          | 0/840790 [00:00<?, ? examples/s]

In [ ]:
!pip install hugginface_hub

In [19]:
data_dict = dataset_prepared.train_test_split(test_size=0.1, shuffle=True)

In [1]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy='epoch' # evaluation strategy to adopt during training
    
)

trainer = Trainer(
    model=model,                         
    args=training_args,
    train_dataset=data_dict['train'],
    eval_dataset=data_dict['test']
)

ModuleNotFoundError: ignored

In [21]:
trainer.train()

/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


OutOfMemoryError: ignored

In [ ]:
import math

eval_result = trainer.evaluate() # returns a dict with the evaluation metrics, cross-entropy loss and perplexity
pprint(eval_result)